# LLM fine-tune code

Author: Alejandro Meza

This notebook demonstrates the full process:
- **1. Loading a base model**
- **2. Applying LoRA adapters for efficient fine-tuning**
- **3. Formatting a custom dataset**
- **4. Training the model with SFTTrainer**
- **5. Running inference to test the fine-tuned LLM**

For this notebook, we are gonna use **Unsloth**. Unsloth is an open-source framework / platform for fine-tuning, training, and reinforcement learning (RL) of large language models (LLMs).  Its goal is to make creating custom LLMs faster, cheaper (in terms of GPU / memory usage), and more accessible to users.

**Acknowledgment**: Special thanks to the YouTube channel Tech with Tim for the video "[EASIEST Way to Fine-Tune a LLM and Use It With Ollama]", which greatly inspired the creation of this notebook. The content of the video served as a great reference.

## JSON -input data- definition

Define a sample dataset to fine-tune our LLM. This time, the sample dataset is gonna be a **JJK** dataset!



In [ ]:
json_demo = [
  {
    "instruction": "Summarize the following Jujutsu Kaisen scene.",
    "input": "During the Shibuya Incident, Satoru Gojo single-handedly fights off numerous cursed spirits and special grade sorcerers inside a crowded station.",
    "output": "Gojo dominates multiple powerful enemies in Shibuya Station during the Shibuya Incident, showcasing his overwhelming strength."
  },
  {
    "instruction": "Translate this Jujutsu Kaisen quote to English.",
    "input": "俺は最強だ。",
    "output": "I am the strongest."
  },
  {
    "instruction": "List three main abilities of Satoru Gojo.",
    "input": "",
    "output": "1. Limitless (infinite manipulation of space)\n2. Six Eyes (enhanced perception and cursed energy control)\n3. Hollow Purple (destructive technique combining Blue and Red)"
  },
  {
    "instruction": "Write a short dialogue between Yuji Itadori and Megumi Fushiguro about training.",
    "input": "",
    "output": "Yuji: \"Megumi, let's spar again! I need to get faster.\"\nMegumi: \"You just fought Sukuna inside your own body. Maybe rest first.\"\nYuji: \"Rest is for the weak!\""
  },
  {
    "instruction": "Classify the following character as sorcerer, curse, or non-sorcerer.",
    "input": "Mahito",
    "output": "curse"
  },
  {
    "instruction": "Describe the Black Flash technique in one sentence.",
    "input": "",
    "output": "Black Flash is a distorted space attack where cursed energy strikes within 0.000001 seconds of a physical hit, amplifying its power."
  },
  {
    "instruction": "Explain the relationship between Yuji Itadori and Sukuna in two sentences.",
    "input": "",
    "output": "Yuji Itadori became Sukuna’s vessel after consuming one of his cursed fingers. Sukuna can manifest within Yuji’s body, creating a constant struggle for control."
  },
  {
    "instruction": "Given the following characters, identify which one possesses the Ten Shadows Technique.",
    "input": "Choices: [Megumi Fushiguro, Nobara Kugisaki, Kento Nanami]",
    "output": "Megumi Fushiguro"
  },
  {
    "instruction": "Write a motivational quote Gojo might say to inspire his students.",
    "input": "",
    "output": "\"Strength isn't just about cursed energy—it's about believing that you can always surpass your limits.\""
  },
  {
    "instruction": "Predict what strategy Megumi would use if facing a stronger opponent alone.",
    "input": "",
    "output": "Megumi would summon multiple shikigami to create openings and strategically retreat while analyzing the opponent’s weaknesses."
  },
  {
    "instruction": "Translate this character name into kanji if possible.",
    "input": "Yuji Itadori",
    "output": "虎杖悠仁"
  },
  {
    "instruction": "Provide a one-sentence description of Nobara Kugisaki’s combat style.",
    "input": "",
    "output": "Nobara fights with a combination of cursed nails, a hammer, and Straw Doll Technique to attack enemies from a distance or inflict resonant damage."
  },
  {
    "instruction": "Summarize the importance of cursed fingers in the story.",
    "input": "",
    "output": "Sukuna’s cursed fingers are powerful remnants that contain fragments of his soul, driving the plot as both sorcerers and curses seek them to gain power or prevent disaster."
  }
]

## Start the code!

### Install the main dependencies

In [ ]:
import torch

In [ ]:
%%capture
!pip install unsloth trl peft accelerate bitsandbytes

### Check running device

In [ ]:
#device check
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

GPU: Tesla T4


### Import model

In [ ]:
from unsloth import FastLanguageModel

# Name of the base model to fine-tune or run inference with  (quantized model of 4 bits)
# You can select other models from here: https://docs.unsloth.ai/get-started/all-our-models
model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit"

# Maximum sequence length (context window) the model will handle during training/inference.
# Increase if your dataset contains longer prompts/responses (must be supported by the model).
max_seq_length = 2048

# Data type for model weights/activations.
# None = let Unsloth/transformers automatically choose the most efficient dtype (fp16/bf16).
dtype = None

# Load both the model and its tokenizer.
# - load_in_4bit=True loads a 4-bit quantized checkpoint to drastically reduce VRAM usage.
# - max_seq_length sets the context window used by the model and tokenizer.
# - dtype selects the precision (auto if None).
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.9.11: Fast Gemma3 patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


### Format our data

In [ ]:
#define a function for format our text
from datasets import Dataset
import json

def format_prompt(example):
    return (
        f"### Instruction:\n{example['instruction']}\n\n"
        f"### Input:\n{example['input']}\n\n"
        f"### Response:\n{example['output']}<|endoftext|>"
    )

formatted_data = [format_prompt(item) for item in json_demo]
dataset = Dataset.from_dict({"text": formatted_data})
print(dataset[0]["text"])

### Instruction:
Summarize the following Jujutsu Kaisen scene.

### Input:
During the Shibuya Incident, Satoru Gojo single-handedly fights off numerous cursed spirits and special grade sorcerers inside a crowded station.

### Response:
Gojo dominates multiple powerful enemies in Shibuya Station during the Shibuya Incident, showcasing his overwhelming strength.<|endoftext|>


### Training

In [ ]:
# To fine-tune a large language model efficiently, we use LoRA (Low-Rank Adaptation) adapters.
# LoRA allows us to train only a small subset of weights while keeping the majority of the model frozen,
# which drastically reduces memory usage and speeds up training.
# LoRA paper: https://arxiv.org/abs/2106.09685

model = FastLanguageModel.get_peft_model(
    model,

    # LoRA rank: determines the dimensionality of the low-rank matrices.
    # Higher rank = closer to full model performance but uses more memory.
    r=64,

    # Target modules in the model where LoRA adapters will be applied.
    # Common modules for transformers: query, key, value, output projections, and feed-forward projections.
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],

    # LoRA scaling factor: often set to 2x rank. Scales the low-rank update during training.
    lora_alpha=132,

    # Dropout applied to LoRA weights during training.
    lora_dropout=0,

    # Bias setting for LoRA layers. "none" is recommended for efficiency.
    bias="none",

    # Use gradient checkpointing to save memory.
    # "unsloth" uses the library’s optimized version for faster backpropagation.
    use_gradient_checkpointing="unsloth",

    # Seed for reproducibility
    random_state=3407,

    # Whether to use Rank-Stabilized LoRA (optional advanced method)
    use_rslora=False,

    # LoftQ configuration (advanced quantization for LoRA, can be left None)
    loftq_config=None,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# ----------------------------
# Define fine-tuning hyperparameters
# ----------------------------
NUM_EPOCHS = 5         # Number of full passes over the dataset
LEARNING_RATE = 2e-4   # Learning rate for optimizer

# ----------------------------
# Set up the SFTTrainer (Supervised Fine-Tuning)
# ----------------------------
trainer = SFTTrainer(
    model=model,                  # The LoRA-adapted model
    tokenizer=tokenizer,          # Tokenizer corresponding to the model
    train_dataset=dataset,        # Dataset to fine-tune on (already formatted)
    dataset_text_field="text",    # Field in dataset containing training text
    max_seq_length=max_seq_length,# Max sequence length for tokenization
    dataset_num_proc=2,           # Number of processes for dataset preprocessing

    args=TrainingArguments(
        # ------------------------
        # Batch & gradient settings
        # ------------------------
        per_device_train_batch_size=2,    # Number of samples per GPU per step
        gradient_accumulation_steps=4,   # Accumulate gradients to simulate larger batch size
                                          # Effective batch size = 2 * 4 = 8

        # ------------------------
        # Training schedule
        # ------------------------
        warmup_steps=10,                 # Linear warmup to stabilize initial training
        num_train_epochs=NUM_EPOCHS,     # Number of epochs
        learning_rate=LEARNING_RATE,     # Optimizer learning rate

        # ------------------------
        # Precision settings
        # ------------------------
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 not available
        bf16=torch.cuda.is_bf16_supported(),     # Use BF16 if supported (faster & memory efficient)

        # ------------------------
        # Logging & optimization
        # ------------------------
        logging_steps=25,                # Log training metrics every N steps
        optim="adamw_8bit",             # Memory-efficient 8-bit AdamW optimizer
        weight_decay=0.01,               # L2 regularization to prevent overfitting
        lr_scheduler_type="linear",      # Linear LR decay

        # ------------------------
        # Reproducibility & saving
        # ------------------------
        seed=3407,                       # Random seed for reproducibility
        output_dir="outputs",            # Directory to save checkpoints
        save_strategy="epoch",           # Save model every epoch
        save_total_limit=2,              # Keep only the 2 most recent checkpoints

        # ------------------------
        # Performance tweaks
        # ------------------------
        dataloader_pin_memory=False,     # Avoid extra pinned memory (optional)
        report_to="none",                # Disable external logging (e.g., Weights & Biases)
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/13 [00:00<?, ? examples/s]

In [ ]:
# ========================== Train the model ====================================
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 13 | Num Epochs = 5 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 52,183,040 of 1,052,068,992 (4.96% trained)


Step,Training Loss


In [ ]:
# ----------------------------
# Enable optimized inference
# ----------------------------
# FastLanguageModel.for_inference enables Unsloth's native optimizations
# which can make generation up to 2x faster on GPU without changing model weights.
FastLanguageModel.for_inference(model)

# ----------------------------
# Define a test prompt
# ----------------------------
# We provide a chat-style input to the fine-tuned model.
# Here, we ask a Jujutsu Kaisen–related question to see if the model learned the dataset.
messages = [
    {
        "role": "user",
        "content": (
            "Explain the relationship between Yuji Itadori and Sukuna in Jujutsu Kaisen?"
        )
    }
]

# ----------------------------
# Tokenize the prompt
# ----------------------------
# apply_chat_template converts chat messages into a single input string for the model.
# - tokenize=True: converts text to input IDs
# - add_generation_prompt=True: appends special tokens that signal model to generate a response
# - return_tensors="pt": returns PyTorch tensors
# Finally, we move tensors to GPU for fast generation.
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# ----------------------------
# Generate response from the fine-tuned model
# ----------------------------
# Parameters:
# - max_new_tokens: maximum tokens to generate
# - use_cache: speed optimization
# - temperature: controls randomness (0.7 = moderately creative)
# - do_sample=True: enables sampling instead of greedy decoding
# - top_p=0.9: nucleus sampling to improve output quality
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# ----------------------------
# Decode and print the response
# ----------------------------
# batch_decode converts token IDs back into readable text.
response = tokenizer.batch_decode(outputs)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<bos><start_of_turn>user
Explain the relationship between Yuji Itadori and Sukuna in Jujutsu Kaisen?<end_of_turn>
<start_of_turn>model
Okay, let’s break down the incredibly complex and central relationship between Yuji Itadori and Sukuna in Jujutsu Kaisen. It’s arguably the most significant and dangerous aspect of his cursed technique. Here’s a layered explanation:

**1. Sukuna’s Manifestation in Yuji’s Body:**

* **The Core Concept:** Sukuna isn’t just cursed energy within Yuji. Instead, Sukuna *manifests within* his own body, particularly in his cursed technique, cursed energy coils. Yuji constantly manipulates his cursed energy to create and control Sukuna’s influence.
* **Yuji’s Cursed Energy Coils:** Yuji creates multiple cursed energy coils around himself – each one representing a different Sukuna fragment. These coils aren’t just for combat; they’re intensely cursed energy that can be manipulated for various effects.


**2. Sukuna’s Influence & Yuji’s Weakness:**

* **Eternal Re

### Save the model

In [ ]:
pip install --upgrade "protobuf<3.21"

In [ ]:
#Save the weights of the model
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

## References

1. [EASIEST Way to Fine-Tune a LLM and Use It With Ollama
]  https://www.youtube.com/watch?v=pTaSDVz0gok&t=1062s